<a href="https://colab.research.google.com/github/isold23/RAG/blob/main/rag-db.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from langchain.llms import LlamaCpp
from langchain import PromptTemplate, LLMChain
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import (
    StreamingStdOutCallbackHandler
)
from langchain.vectorstores import FAISS
from langchain.vectorstores import Milvus
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

import os
import sys

template = """Question: {question}

Answer:"""

prompt = PromptTemplate(template=template, input_variables=["question"])
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

print("sys.path[0] = ", sys.path[0])
#os.chdir("/content/drive/MyDrive/")


llm = LlamaCpp(
    model_path="/content/drive/MyDrive/llama-2-7b-chat.ggmlv3.q2_K.bin",
    #model_path="/drive/MyDrive/llama-2-13b-chat.ggmlv3.q4_0.bin",
    n_ctx=6000,
    n_gpu_layers=512,
    n_batch=30,
    callback_manager=callback_manager,
    temperature = 0.9,
    max_tokens = 4095,
    n_parts=1,
)

llm_chain = LLMChain(prompt=prompt, llm=llm)


#loader = UnstructuredFileLoader("AI discovers over 20K taxable French swimming pools.docx")
loader = UnstructuredFileLoader("/content/drive/MyDrive/lsmtree.pdf")

documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
docs = text_splitter.split_documents(documents)

# embedding engine
hf_embedding = HuggingFaceInstructEmbeddings()

#db = FAISS.from_documents(docs, hf_embedding)

# save embeddings in local directory
#db.save_local("faiss_AiArticle")

# load from local
#db = FAISS.load_local("faiss_AiArticle/", embeddings=hf_embedding)


db = Milvus.from_documents(
 docs,
 hf_embedding,
 connection_args={"host": "127.0.0.1", "port": "19530"},
 collection_name = "clinical_trial_langchain", ## custom collection name
 search_params = {"metric":"IP","offset":0}, ## search params
)

#query = "why french government using AI?"
query = "Introducing lsm tree"
search = db.similarity_search(query, k=2)

print(search)

sys.path[0] =  /content


ValidationError: ignored

In [ ]:
!ls -la ./drive/MyDrive

total 2799745
drwx------ 2 root root       4096 Sep 28 06:39 'Colab Notebooks'
drwx------ 2 root root       4096 Sep 28 09:15  faiss_AiArticle
-rw------- 1 root root 2866807424 Sep 27 00:42  llama-2-7b-chat.ggmlv3.q2_K.bin
-rw------- 1 root root     122526 Sep 28 06:35  lsmtree.pdf


In [ ]:
!ps auxf |grep milvus

root        4985  0.0  0.0   7372  3544 ?        S    07:54   0:00  |       \_ /bin/bash -c ps auxf |grep milvus
root        4987  0.0  0.0   6480  2248 ?        S    07:54   0:00  |           \_ grep milvus


In [ ]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.0 MB/s eta 0:00:00
